# Week 2: arXiv API Integration & PDF Processing

**What We're Building This Week:**

Week 2 focuses on implementing the core data ingestion pipeline that will automatically fetch, process, and store arXiv papers. This is the foundation that feeds our RAG system with fresh academic content.

## Week 2 Focus Areas

### 🎯 Core Objectives
- **arXiv API Integration**: Build a robust client with rate limiting and retry logic
- **PDF Processing Pipeline**: Download and parse scientific PDFs with structured content extraction
- **Database Storage**: Persist paper metadata and content in PostgreSQL
- **Error Handling**: Implement comprehensive error handling and graceful degradation
- **Automation Ready**: Prepare components for Airflow orchestration

### 🔧 What We'll Test In This Notebook
1. **arXiv API Client** - Fetch CS.AI papers with proper rate limiting
2. **PDF Download System** - Download and cache PDFs with error handling  
3. **Docling PDF Parser** - Extract structured content (sections, tables, figures)
4. **Database Integration** - Store and retrieve papers from PostgreSQL
5. **Complete Pipeline** - End-to-end processing from arXiv to database
6. **Production Readiness** - Error handling, logging, and performance metrics


### 📊 Success Metrics
- arXiv API calls succeed with proper rate limiting
- PDF download and caching works reliably  
- Docling extracts structured content from scientific PDFs
- Database stores complete paper metadata
- Pipeline handles errors gracefully and continues processing
- All components ready for Airflow automation (Week 2+)

---

## Week 2 Component Status
| Component | Purpose | Status |
|-----------|---------|--------|
| **arXiv API Client** | Fetch CS.AI papers with rate limiting | ✅ Complete |
| **PDF Downloader** | Download and cache PDFs locally | ✅ Complete |
| **Docling Parser** | Extract structured content from PDFs | ✅ Complete |
| **Metadata Fetcher** | Orchestrate complete pipeline | ✅ Complete |
| **Database Storage** | Store papers in PostgreSQL | ⚠️ Needs volume refresh |
| **Airflow DAGs** | Automate daily ingestion | ⚠️ Needs container update |


## ⚠️ IMPORTANT: Week 2 Database Schema Update

**NEW USERS OR SCHEMA CONFLICTS**: If you're starting Week 2 fresh or experiencing database schema conflicts, use this clean start approach:

### Fresh Start (Recommended for Week 2)
```bash
# Complete clean slate - removes all data but ensures correct schema
docker compose down -v

# Build fresh containers with latest code
docker compose up --build -d
```

**When to use this:**
- First time running Week 2 
- Schema errors or column missing errors
- Want to start with clean database
- Previous Week 1 data not important

**Note**: This destroys existing data but ensures you have the correct Week 2 schema with all new columns for PDF processing and arXiv metadata.

---

## Prerequisites Check

**Before starting:**
1. Week 1 infrastructure completed
2. UV environment activated
3. Docker Desktop running

**Why fresh containers?** Week 2 includes new Airflow dependencies and code changes that require rebuilding images rather than using cached layers.

In [37]:
# Check if Fresh Containers are Built and All Services Healthy
import subprocess
import requests
from pathlib import Path

print("WEEK 2 CONTAINER & SERVICE HEALTH CHECK")
print("=" * 50)

# Find project root
current_dir = Path.cwd()
if current_dir.name == "week2" and current_dir.parent.name == "notebooks":
    project_root = current_dir.parent.parent
elif (current_dir / "compose.yml").exists():
    project_root = current_dir
else:
    print("✗ Could not find project root")
    exit()

print(f"Project root: {project_root}")

# Step 1: Check if containers are built and running
print("\n1. Checking container status...")
try:
    result = subprocess.run(
        ["docker", "compose", "ps", "--format", "table"],
        cwd=str(project_root),
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0 and result.stdout.strip():
        print("✓ Containers are running:")
        for line in result.stdout.strip().split('\n'):
            print(f"   {line}")
    else:
        print("✗ No containers running or docker compose failed")
        print("Please run the build commands from the markdown cell above")
        exit()
        
except Exception as e:
    print(f"✗ Error checking containers: {e}")
    print("Please run the build commands from the markdown cell above")
    exit()

# Step 2: Check all service health (corrected endpoints)
print("\n2. Checking service health...")
services_to_test = {
    "FastAPI": "http://localhost:8000/api/v1/health",
    "PostgreSQL (via API)": "http://localhost:8000/api/v1/health", 
    "Ollama": "http://localhost:11434/api/version",
    "OpenSearch": "http://localhost:9200/_cluster/health",
    "Airflow": "http://localhost:8080/health"
}

all_healthy = True
for service_name, url in services_to_test.items():
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✓ {service_name}: Healthy")
        else:
            print(f"✗ {service_name}: HTTP {response.status_code}")
            all_healthy = False
    except requests.exceptions.ConnectionError:
        print(f"✗ {service_name}: Not accessible")
        all_healthy = False
    except Exception as e:
        print(f"✗ {service_name}: {type(e).__name__}")
        all_healthy = False

print("\n" + "=" * 50)
if all_healthy:
    print("✓ ALL SERVICES HEALTHY! Ready for Week 2 development.")
else:
    print("✗ Some services need attention.")
    print("If you just rebuilt containers, wait 1-2 minutes and run this cell again.")
    print("Airflow and OpenSearch take longest to start up.")

WEEK 2 CONTAINER & SERVICE HEALTH CHECK
Project root: d:\Projects\Agentic_RAG\arxiv-paper-curator

1. Checking container status...
✓ Containers are running:
   NAME             IMAGE                                            COMMAND                  SERVICE                 CREATED       STATUS                 PORTS
   rag-airflow      arxiv-paper-curator-airflow                      "/entrypoint.sh"         airflow                 4 hours ago   Up 4 hours (healthy)   0.0.0.0:8080->8080/tcp, [::]:8080->8080/tcp
   rag-api          arxiv-paper-curator-api                          "uvicorn src.main:apâ€¦"   api                     4 hours ago   Up 4 hours (healthy)   0.0.0.0:8000->8000/tcp, [::]:8000->8000/tcp
   rag-dashboards   opensearchproject/opensearch-dashboards:2.19.0   "./opensearch-dashboâ€¦"   opensearch-dashboards   4 hours ago   Up 4 hours (healthy)   0.0.0.0:5601->5601/tcp, [::]:5601->5601/tcp
   rag-ollama       ollama/ollama:0.11.2                             "/bin/ollama

In [38]:
# Environment Check
import sys
from pathlib import Path

print(f"Python Version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"Environment: {sys.executable}")

# Find project root
current_dir = Path.cwd()
if current_dir.name == "week2" and current_dir.parent.name == "notebooks":
    project_root = current_dir.parent.parent
elif (current_dir / "compose.yml").exists():
    project_root = current_dir
else:
    project_root = None

if project_root and (project_root / "compose.yml").exists():
    print(f"✓ Project root: {project_root}")
    # Add project to Python path
    sys.path.insert(0, str(project_root))
else:
    print("✗ Missing compose.yml - check directory")
    exit()

Python Version: 3.12.12
Environment: d:\Projects\Agentic_RAG\arxiv-paper-curator\.venv\Scripts\python.exe
✓ Project root: d:\Projects\Agentic_RAG\arxiv-paper-curator


## Service Health Verification

Ensure all services from Week 1 are still running correctly:

### 🔗 Service Access Points
- **FastAPI**: http://localhost:8000/docs (API documentation)
- **PostgreSQL**: via API or `docker exec -it rag-postgres psql -U rag_user -d rag_db`
- **OpenSearch**: http://localhost:9200/_cluster/health
- **Ollama**: http://localhost:11434 (LLM service)
- **Airflow**: http://localhost:8080 (Username: `admin`, Password: `admin`)

In [39]:
# Test Service Connectivity
import requests
import subprocess
import json

services_to_test = {
    "FastAPI": "http://localhost:8000/api/v1/health",
    "PostgreSQL (via API)": "http://localhost:8000/api/v1/health", 
    "Ollama": "http://localhost:11434/api/version",
    "OpenSearch": "http://localhost:9200/_cluster/health",
    "Airflow": "http://localhost:8080/health"  
}

print("WEEK 2 PREREQUISITE CHECK")
print("=" * 50)

all_healthy = True

for service_name, url in services_to_test.items():
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✓ {service_name}: Healthy")
        else:
            print(f"✗ {service_name}: HTTP {response.status_code}")
            all_healthy = False
    except requests.exceptions.ConnectionError:
        print(f"✗ {service_name}: Not accessible")
        all_healthy = False
    except Exception as e:
        print(f"✗ {service_name}: {type(e).__name__}")
        all_healthy = False

print()
if all_healthy:
    print("All services healthy! Ready for Week 2 development.")
else:
    print("Some services need attention. Check Week 1 notebook.")

WEEK 2 PREREQUISITE CHECK
✓ FastAPI: Healthy
✓ PostgreSQL (via API): Healthy
✓ Ollama: Healthy
✓ OpenSearch: Healthy
✓ Airflow: Healthy

All services healthy! Ready for Week 2 development.


## 1. arXiv API Client Testing

Test the arXiv API client with rate limiting and retry logic:

In [40]:
# Test arXiv API Client
import asyncio
from datetime import datetime, timedelta
import importlib
import os

# Debug: Check what the OLLAMA_MODELS environment variable is set to
if 'OLLAMA_MODELS' in os.environ:
    print(f"DEBUG: OLLAMA_MODELS = '{os.environ['OLLAMA_MODELS']}' (repr: {repr(os.environ['OLLAMA_MODELS'])})")
    # Delete it completely to avoid JSON parsing issues
    del os.environ['OLLAMA_MODELS']
    print("Cleared OLLAMA_MODELS environment variable")

# Reload config module to get the latest changes
import src.config
importlib.reload(src.config)

# Import our arXiv client
from src.services.arxiv.factory import make_arxiv_client

print("\nTESTING ARXIV API CLIENT")
print("=" * 40)

# Create client
arxiv_client = make_arxiv_client()
print(f"✓ Client created: {arxiv_client.base_url}")
print(f"   Rate limit: {arxiv_client.rate_limit_delay}s")
print(f"   Max results: {arxiv_client.max_results}")
print(f"   Category: {arxiv_client.search_category}")
print()


TESTING ARXIV API CLIENT
✓ Client created: https://export.arxiv.org/api/query
   Rate limit: 3.0s
   Max results: 100
   Category: cs.AI



In [41]:
# Test Paper Fetching
async def test_paper_fetching():
    """Test fetching papers from arXiv with rate limiting."""
    
    print("Test 1: Fetch Recent CS.AI Papers")
    try:
        papers = await arxiv_client.fetch_papers(
            max_results=2, 
            sort_by="submittedDate",
            sort_order="descending"
        )
        
        print(f"✓ Fetched {len(papers)} papers")
        
        if papers:
            for i, paper in enumerate(papers[:2], 1):
                print(f"   {i}. [{paper.arxiv_id}] {paper.title[:60]}...")
                print(f"      Authors: {', '.join(paper.authors[:2])}{'...' if len(paper.authors) > 2 else ''}")
                print(f"      Categories: {', '.join(paper.categories)}")
                print(f"      Published: {paper.published_date}")
                print()
        
        return papers
        
    except Exception as e:
        print(f"✗ Error fetching papers: {e}")
        if "503" in str(e):
            print("   arXiv API temporarily unavailable (normal)")
            print("   Rate limiting and error handling working correctly")
        return []

# Run the test
papers = await test_paper_fetching()

Test 1: Fetch Recent CS.AI Papers
✓ Fetched 2 papers
   1. [2601.00791v1] Geometry of Reason: Spectral Signatures of Valid Mathematica...
      Authors: Valentin Noël
      Categories: cs.LG, cs.AI, cs.CL, cs.LO
      Published: 2026-01-02T18:49:37Z

   2. [2601.00785v1] FedHypeVAE: Federated Learning with Hypernetwork Generated C...
      Authors: Sunny Gupta, Amit Sethi
      Categories: cs.LG, cs.AI, cs.CV
      Published: 2026-01-02T18:40:41Z



In [42]:
# Test Date Filtering
async def test_date_filtering():
    """Test date range filtering functionality."""
    
    print("Test 2: Date Range Filtering")
    
    # Use specific dates: 
    from_date = "20250808"  
    to_date = "20250809"    
    try:
        date_papers = await arxiv_client.fetch_papers(
            max_results=5,
            from_date=from_date,
            to_date=to_date
        )
        
        print(f"✓ Date filtering test: {len(date_papers)} papers from {from_date}-{to_date}")
        
        if date_papers:
            for i, paper in enumerate(date_papers, 1):
                print(f"   {i}. [{paper.arxiv_id}] {paper.title[:60]}...")
                print(f"      Authors: {', '.join(paper.authors[:2])}{'...' if len(paper.authors) > 2 else ''}")
                print(f"      Categories: {', '.join(paper.categories)}")
                print(f"      Published: {paper.published_date}")
                print()
        
        return date_papers
        
    except Exception as e:
        print(f"✗ Date filtering error: {e}")
        return []

# Run date filtering test
date_papers = await test_date_filtering()

Test 2: Date Range Filtering
✓ Date filtering test: 5 papers from 20250808-20250809
   1. [2508.07111v1] Investigating Intersectional Bias in Large Language Models u...
      Authors: Falaah Arif Khan, Nivedha Sivakumar...
      Categories: cs.CL, cs.AI
      Published: 2025-08-09T22:24:40Z

   2. [2508.07107v2] Designing a Feedback-Driven Decision Support System for Dyna...
      Authors: Timothy Oluwapelumi Adeyemi, Nadiah Fahad AlOtaibi
      Categories: cs.AI, cs.CY
      Published: 2025-08-09T21:24:54Z

   3. [2508.07102v1] Towards High-Order Mean Flow Generative Models: Feasibility,...
      Authors: Yang Cao, Yubin Chen...
      Categories: cs.LG, cs.AI, cs.CV
      Published: 2025-08-09T21:10:58Z

   4. [2508.07101v1] Less Is More: Training-Free Sparse Attention with Global Loc...
      Authors: Lijie Yang, Zhihao Zhang...
      Categories: cs.CL, cs.AI
      Published: 2025-08-09T21:10:33Z

   5. [2508.07095v1] Hide or Highlight: Understanding the Impact of Factuality Ex...
  

## 2. PDF Download and Caching

Test PDF download functionality with caching:

In [43]:
# Test PDF Download
async def test_pdf_download(test_papers):
    """Test PDF downloading with caching."""

    print("Test 3: PDF Download & Caching")
    
    if not test_papers:
        print("No papers available for PDF download test")
        return None
    
    # Test with first paper
    test_paper = test_papers[0]
    print(f"Testing PDF download for: {test_paper.arxiv_id}")
    print(f"Title: {test_paper.title[:60]}...")
    
    try:
        # Download PDF 
        pdf_path = await arxiv_client.download_pdf(test_paper)
        
        if pdf_path and pdf_path.exists():
            size_mb = pdf_path.stat().st_size / (1024 * 1024)
            print(f"✓ PDF downloaded: {pdf_path.name} ({size_mb:.2f} MB)")
            
            return pdf_path
        else:
            print("✗ PDF download failed")
            return None
            
    except Exception as e:
        print(f"✗ PDF download error: {e}")
        return None

# Run PDF download test 
pdf_path = await test_pdf_download(date_papers[:1])

Test 3: PDF Download & Caching
Testing PDF download for: 2508.07111v1
Title: Investigating Intersectional Bias in Large Language Models u...
✓ PDF downloaded: 2508.07111v1.pdf (6.81 MB)


## 3. Docling PDF Processing

Test PDF parsing with Docling for structured content extraction:

In [44]:
# Test PDF Parsing with Docling
from src.services.pdf_parser.factory import make_pdf_parser_service
from src.config import get_settings
from pathlib import Path

print("Test 4: PDF Parsing with Docling")
print("=" * 40)

# Create PDF parser
pdf_parser = make_pdf_parser_service()
settings = get_settings()
print("PDF parser service created")
print(f"Config: {settings.pdf_parser.max_pages} pages, {settings.pdf_parser.max_file_size_mb}MB")

# Test parsing with actual PDF files
cache_dir = Path("data/arxiv_pdfs")
if cache_dir.exists():
    pdf_files = list(cache_dir.glob("*.pdf"))
    print(f"\nFound {len(pdf_files)} PDF files to test parsing")
    
    if pdf_files:
        # Test parsing the first PDF
        test_pdf = pdf_files[0]
        print(f"Testing PDF parsing with: {test_pdf.name}")
        
        try:
            pdf_content = await pdf_parser.parse_pdf(test_pdf)
            
            if pdf_content:
                print(f"✓ PDF parsing successful!")
                print(f"  Sections: {len(pdf_content.sections)}")
                print(f"  Raw text length: {len(pdf_content.raw_text)} characters")
                print(f"  Parser used: {pdf_content.parser_used}")
                
                # Show first section as example
                if pdf_content.sections:
                    first_section = pdf_content.sections[0]
                    print(f"  First section: '{first_section.title}' ({len(first_section.content)} chars)")
            else:
                print("✗ PDF parsing failed (Docling compatibility issue)")
                print("This is expected - not all PDFs work with Docling")
                
        except Exception as e:
            print(f"✗ PDF parsing error: {e}")
            print("This demonstrates the error handling in action")
    else:
        print("No PDF files available for parsing test")
else:
    print("No PDF cache directory found")

Test 4: PDF Parsing with Docling
PDF parser service created
Config: 30 pages, 20MB

Found 4 PDF files to test parsing
Testing PDF parsing with: 2508.07111v1.pdf


Parameter `strict_text` has been deprecated and will be ignored.


✓ PDF parsing successful!
  Sections: 23
  Raw text length: 100641 characters
  Parser used: ParserType.DOCLING
  First section: 'Content' (84 chars)


## 4. Database Storage Testing

Test storing papers in PostgreSQL database:

In [45]:
# Test Database Storage
from src.db.factory import make_database
from src.repositories.paper import PaperRepository
from src.schemas.arxiv.paper import PaperCreate
from dateutil import parser as date_parser

print("Test 5: Database Storage")
print("=" * 40)

# Create database connection
database = make_database()
print("✓ Database connection created")

if papers:
    test_paper = papers[0]
    print(f"Storing paper: {test_paper.arxiv_id}")
    
    try:
        with database.get_session() as session:
            paper_repo = PaperRepository(session)
            
            # Convert to database format
            published_date = date_parser.parse(test_paper.published_date) if isinstance(test_paper.published_date, str) else test_paper.published_date
            
            paper_create = PaperCreate(
                arxiv_id=test_paper.arxiv_id,
                title=test_paper.title,
                authors=test_paper.authors,
                abstract=test_paper.abstract,
                categories=test_paper.categories,
                published_date=published_date,
                pdf_url=test_paper.pdf_url
            )
            
            # Store paper (upsert to avoid duplicates)
            stored_paper = paper_repo.upsert(paper_create)
            
            if stored_paper:
                print(f"✓ Paper stored with ID: {stored_paper.id}")
                print(f"   Database ID: {stored_paper.id}")
                print(f"   arXiv ID: {stored_paper.arxiv_id}")
                print(f"   Title: {stored_paper.title[:50]}...")
                print(f"   Authors: {len(stored_paper.authors)} authors")
                print(f"   Categories: {', '.join(stored_paper.categories)}")
                
                # Test retrieval
                retrieved_paper = paper_repo.get_by_arxiv_id(test_paper.arxiv_id)
                if retrieved_paper:
                    print(f"✓ Paper retrieval test passed")
                else:
                    print(f"✗ Paper retrieval failed")
            else:
                print("✗ Paper storage failed")
                
    except Exception as e:
        print(f"✗ Database error: {e}")
else:
    print("No papers available for database storage test")

Test 5: Database Storage
✓ Database connection created
Storing paper: 2601.00791v1
✓ Paper stored with ID: 0bf699c9-9c39-4cd8-a71c-c30d965d0ae6
   Database ID: 0bf699c9-9c39-4cd8-a71c-c30d965d0ae6
   arXiv ID: 2601.00791v1
   Title: Geometry of Reason: Spectral Signatures of Valid M...
   Authors: 1 authors
   Categories: cs.LG, cs.AI, cs.CL, cs.LO
✓ Paper retrieval test passed


In [46]:
# Method 4: Database Statistics
from sqlalchemy import text

with database.get_session() as session:
    stats_query = text("""
        SELECT 
            COUNT(DISTINCT p.id) as total_papers,
            COUNT(DISTINCT category) as unique_categories,
            AVG(json_array_length(p.authors)) as avg_authors,
            MIN(p.published_date) as earliest_paper,
            MAX(p.published_date) as latest_paper,
            COUNT(DISTINCT CASE WHEN p.pdf_processing_date IS NOT NULL THEN p.id END) as parsed_pdfs
        FROM papers p
        CROSS JOIN LATERAL json_array_elements_text(p.categories) as category
    """)
    
    result = session.execute(stats_query)
    stats = result.fetchone()
    
    print("DATABASE STATISTICS")
    print("=" * 50)
    if stats and stats[0] > 0:
        print(f"Total Papers: {stats[0]}")
        print(f"Unique Categories: {stats[1]}")
        print(f"Average Authors per Paper: {stats[2]:.1f}")
        print(f"Date Range: {stats[3]} to {stats[4]}")
        print(f"Parsed PDFs: {stats[5]}")
    else:
        print("No papers in database yet")
    print("=" * 50)

DATABASE STATISTICS
Total Papers: 15
Unique Categories: 14
Average Authors per Paper: 3.4
Date Range: 2025-08-14 23:27:09 to 2026-01-02 18:49:37
Parsed PDFs: 3


In [32]:
# Method 3: Use psql in Docker (Command Line)
print("METHOD 3: Access Database via psql CLI")
print("=" * 50)
print("\nYou can also access the database using psql in the Docker container:")
print("\n1. Open a terminal/PowerShell")
print("2. Run this command:")
print("   docker exec -it rag-postgres psql -U rag_user -d rag_db")
print("\n3. Then you can run SQL queries like:")
print("   SELECT COUNT(*) FROM papers;")
print("   SELECT arxiv_id, title FROM papers LIMIT 5;")
print("   \\dt  -- List all tables")
print("   \\d papers  -- Describe papers table structure")
print("   \\q  -- Quit psql")
print("\n" + "=" * 50)

METHOD 3: Access Database via psql CLI

You can also access the database using psql in the Docker container:

1. Open a terminal/PowerShell
2. Run this command:
   docker exec -it rag-postgres psql -U rag_user -d rag_db

3. Then you can run SQL queries like:
   SELECT COUNT(*) FROM papers;
   SELECT arxiv_id, title FROM papers LIMIT 5;
   \dt  -- List all tables
   \d papers  -- Describe papers table structure
   \q  -- Quit psql



In [48]:
# Method 2: View Detailed Paper Information
def view_paper_details(arxiv_id: str):
    """View complete details of a specific paper."""
    with database.get_session() as session:
        paper_repo = PaperRepository(session)
        paper = paper_repo.get_by_arxiv_id(arxiv_id)
        
        if paper:
            print(f"PAPER DETAILS: {arxiv_id}")
            print("=" * 70)
            print(f"Database ID: {paper.id}")
            print(f"Title: {paper.title}")
            print(f"\nAuthors ({len(paper.authors)}):")
            for i, author in enumerate(paper.authors, 1):
                print(f"  {i}. {author}")
            print(f"\nCategories: {', '.join(paper.categories)}")
            print(f"Published: {paper.published_date}")
            print(f"PDF URL: {paper.pdf_url}")
            print(f"\nAbstract:\n{paper.abstract}")
            print(f"\nMetadata:")
            print(f"  Created: {paper.created_at}")
            print(f"  Updated: {paper.updated_at}")
            if paper.pdf_processing_date:
                print(f"  PDF Parsed: {paper.pdf_processing_date}")
        else:
            print(f"Paper {arxiv_id} not found in database")

# Example: View the first paper we stored
if papers:
    view_paper_details(papers[0].arxiv_id)

PAPER DETAILS: 2601.00791v1
Database ID: 0bf699c9-9c39-4cd8-a71c-c30d965d0ae6
Title: Geometry of Reason: Spectral Signatures of Valid Mathematical Reasoning

Authors (1):
  1. Valentin Noël

Categories: cs.LG, cs.AI, cs.CL, cs.LO
Published: 2026-01-02 18:49:37
PDF URL: https://arxiv.org/pdf/2601.00791v1

Abstract:
We present a training-free method for detecting valid mathematical reasoning in large language models through spectral analysis of attention patterns. By treating attention matrices as adjacency matrices of dynamic graphs over tokens, we extract four interpretable spectral diagnostics, the Fiedler value (algebraic connectivity), high-frequency energy ratio (HFER), graph signal smoothness, and spectral entropy, that exhibit statistically significant differences between valid and invalid mathematical proofs. Experiments across seven transformer models from four independent architectural families (Meta Llama, Alibaba Qwen, Microsoft Phi, and Mistral AI) demonstrate that this spe

In [49]:
# Method 1: Query Database from Notebook
from sqlalchemy import text
import pandas as pd

print("DATABASE CONTENTS VIEWER")
print("=" * 50)

with database.get_session() as session:
    # Count total papers
    count_result = session.execute(text("SELECT COUNT(*) FROM papers"))
    total_papers = count_result.scalar()
    print(f"\n📊 Total papers in database: {total_papers}")
    
    # Get recent papers
    query = text("""
        SELECT 
            id,
            arxiv_id,
            title,
            json_array_length(authors) as author_count,
            json_array_length(categories) as category_count,
            published_date,
            created_at
        FROM papers
        ORDER BY created_at DESC
        LIMIT 10
    """)
    
    result = session.execute(query)
    papers_df = pd.DataFrame(result.fetchall(), columns=result.keys())
    
    if not papers_df.empty:
        print(f"\n📄 Most Recent {len(papers_df)} Papers:")
        print("-" * 50)
        for idx, row in papers_df.iterrows():
            print(f"\n{idx + 1}. [{row['arxiv_id']}]")
            print(f"   Title: {row['title'][:70]}...")
            print(f"   Authors: {row['author_count']} | Categories: {row['category_count']}")
            print(f"   Published: {row['published_date']} | Stored: {row['created_at']}")
    else:
        print("\nNo papers found in database")

print("\n" + "=" * 50)

DATABASE CONTENTS VIEWER

📊 Total papers in database: 15

📄 Most Recent 10 Papers:
--------------------------------------------------

1. [2508.11110v1]
   Title: Diffusion is a code repair operator and generator...
   Authors: 4 | Categories: 3
   Published: 2025-08-14 23:27:09 | Stored: 2026-01-05 13:55:00.797255

2. [2508.11112v1]
   Title: Quantization through Piecewise-Affine Regularization: Optimization and...
   Authors: 2 | Categories: 4
   Published: 2025-08-14 23:35:21 | Stored: 2026-01-05 13:55:00.735672

3. [2508.11121v1]
   Title: Tabularis Formatus: Predictive Formatting for Tables...
   Authors: 5 | Categories: 3
   Published: 2025-08-14 23:54:40 | Stored: 2026-01-05 13:55:00.709862

4. [2601.00426v1]
   Title: RMAAT: Astrocyte-Inspired Memory Compression and Replay for Efficient ...
   Authors: 3 | Categories: 4
   Published: 2026-01-01 18:34:06 | Stored: 2026-01-05 13:49:48.959193

5. [2601.00448v1]
   Title: Language as Mathematical Structure: Examining Semantic Field

## 📊 Viewing Database Contents

You can view your PostgreSQL database in several ways:

In [28]:
# Test Complete Pipeline
from src.services.metadata_fetcher import make_metadata_fetcher

print("Test 6: Complete Metadata Fetcher Pipeline")
print("=" * 50)

# Create metadata fetcher
metadata_fetcher = make_metadata_fetcher(arxiv_client, pdf_parser)
print("✓ Metadata fetcher service created")

# Test with small batch
print("Running small batch test (2 papers, no PDF processing for speed)...")

try:
    with database.get_session() as session:
        results = await metadata_fetcher.fetch_and_process_papers(
            max_results=2,  
            process_pdfs=False,  
            store_to_db=True,
            db_session=session
        )
    
    print("\nPIPELINE RESULTS:")
    print(f"   Papers fetched: {results.get('papers_fetched', 0)}")
    print(f"   PDFs downloaded: {results.get('pdfs_downloaded', 0)}")
    print(f"   PDFs parsed: {results.get('pdfs_parsed', 0)}")
    print(f"   Papers stored: {results.get('papers_stored', 0)}")
    print(f"   Processing time: {results.get('processing_time', 0):.1f}s")
    print(f"   Errors: {len(results.get('errors', []))}")
    
    if results.get('errors'):
        print("\nErrors encountered:")
        for error in results.get('errors', [])[:3]:  # Show first 3 errors
            print(f"   - {error}")
    
    if results.get('papers_fetched', 0) > 0:
        print("\n✓ Pipeline test successful!")
    else:
        print("\nNo papers fetched - may be arXiv API unavailability")
        
except Exception as e:
    print(f"✗ Pipeline error: {e}")

Test 6: Complete Metadata Fetcher Pipeline
✓ Metadata fetcher service created
Running small batch test (2 papers, no PDF processing for speed)...

PIPELINE RESULTS:
   Papers fetched: 2
   PDFs downloaded: 0
   PDFs parsed: 0
   Papers stored: 2
   Processing time: 0.1s
   Errors: 0

✓ Pipeline test successful!


In [30]:
# Test Airflow DAGs
print("Test 7: Airflow DAG Status")
print("=" * 40)

print("  Airflow UI Access:")
print("   URL: http://localhost:8080")
print("   Username: admin")
print("   Password: admin")
print()

# Check DAG status using docker exec
try:
    result = subprocess.run(
        ["docker", "exec", "rag-airflow", "airflow", "dags", "list"],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        dag_lines = [line for line in lines if 'arxiv' in line.lower() or 'hello' in line.lower()]
        
        print("Available DAGs:")
        for line in dag_lines:
            if '|' in line:
                parts = [part.strip() for part in line.split('|')]
                if len(parts) >= 3:
                    dag_id = parts[0]
                    is_paused = parts[2]
                    status = "Active" if is_paused == "False" else "Paused"
                    print(f"   - {dag_id}: {status}")
        
        # Check for import errors
        error_result = subprocess.run(
            ["docker", "exec", "rag-airflow", "airflow", "dags", "list-import-errors"],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if "docling" in error_result.stderr:
            print("\nKnown Issue: Docling not installed in Airflow container")
            print("   - This is expected for Week 2")
            print("   - DAG structure is complete, runtime needs container fix")
            print("   - Solution: Add docling to Airflow container startup")
        elif error_result.returncode == 0:
            print("\n✓ No DAG import errors found")
        
    else:
        print(f"✗ Could not list DAGs: {result.stderr}")
        
except Exception as e:
    print(f"✗ Airflow test error: {e}")

print("\n  To view DAGs graphically:")
print("   1. Open http://localhost:8080 in your browser")
print("   2. Login with admin/admin")
print("   3. Click on 'arxiv_paper_ingestion' DAG to see the workflow")

Test 7: Airflow DAG Status
  Airflow UI Access:
   URL: http://localhost:8080
   Username: admin
   Password: admin

Available DAGs:
   - arxiv_paper_ingestion: Paused
   - hello_world_week1: Paused

✓ No DAG import errors found

  To view DAGs graphically:
   1. Open http://localhost:8080 in your browser
   2. Login with admin/admin
   3. Click on 'arxiv_paper_ingestion' DAG to see the workflow


In [31]:
# Test Complete Pipeline with PDF Processing
print("Test 8: Complete Pipeline with PDF Processing")
print("=" * 50)

# Reuse metadata fetcher from Test 6
print("✓ Using metadata fetcher service from previous test")

# Test with small batch including PDF processing
print("Running enhanced test (3 papers with PDF processing)...")

try:
    with database.get_session() as session:
        results = await metadata_fetcher.fetch_and_process_papers(
            max_results=3,  # Small batch
            from_date="20250813",  # Recent date
            to_date="20250814",
            process_pdfs=True,  
            store_to_db=True,
            db_session=session
        )
    
    print("\nENHANCED PIPELINE RESULTS:")
    print(f"   Papers fetched: {results.get('papers_fetched', 0)}")
    print(f"   PDFs downloaded: {results.get('pdfs_downloaded', 0)}")
    print(f"   PDFs parsed: {results.get('pdfs_parsed', 0)}")
    print(f"   Papers stored: {results.get('papers_stored', 0)}")
    print(f"   Processing time: {results.get('processing_time', 0):.1f}s")
    print(f"   Errors: {len(results.get('errors', []))}")
    
    # Show success rates
    if results.get('papers_fetched', 0) > 0:
        download_rate = (results['pdfs_downloaded'] / results['papers_fetched']) * 100
        parse_rate = (results['pdfs_parsed'] / results['pdfs_downloaded']) * 100 if results.get('pdfs_downloaded', 0) > 0 else 0
        print(f"   Download success rate: {download_rate:.1f}%")
        print(f"   Parse success rate: {parse_rate:.1f}%")
    
    if results.get('errors'):
        print("\nErrors encountered (showing graceful error handling):")
        for error in results.get('errors', [])[:3]:  # Show first 3 errors
            print(f"   - {error}")
    
    if results.get('papers_fetched', 0) > 0:
        print("\n✓ Enhanced pipeline test successful!")
        if results.get('errors'):
            print("✓ System continued processing despite PDF failures")
    else:
        print("\n! No papers fetched - may be arXiv API unavailability")
        
except Exception as e:
    print(f"✗ Pipeline error: {e}")

Test 8: Complete Pipeline with PDF Processing
✓ Using metadata fetcher service from previous test
Running enhanced test (3 papers with PDF processing)...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\Projects\Agentic_RAG\arxiv-paper-curator\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91820\.cache\huggingface\hub\models--ds4sd--docling-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, se


ENHANCED PIPELINE RESULTS:
   Papers fetched: 3
   PDFs downloaded: 3
   PDFs parsed: 3
   Papers stored: 3
   Processing time: 189.8s
   Errors: 0
   Download success rate: 100.0%
   Parse success rate: 100.0%

✓ Enhanced pipeline test successful!
